In [1]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar. This needs to be the same as the name used in **Part 1** for the files from that part to be loaded.

In [2]:
import panel as pn
pn.extension()
import sys
import os
sys.path.append('/home/notebooks/dashboards-inline')
LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/DEMO


## Part Two: Effects of Shims on the FID Signal and Spectrum

### 1. Overview

In this part we will explore the effects of shims on the FID signal and spectrum. The initial shims will be loaded from your user data directory (displayed above), which should contain a `shims.yaml` file generated by running the autoshim tool in **Part 1**. 

### 2. Theory

TODO

### 3. Manual Shim Adjustment

The dashboard below allows you to modify the shim values and see the effect on the FID signal and spectrum. Try adjusting the shim values in increments of 0.01, which can be done with the scroll wheel after clicking inside the input box, and running the experiment. *Run Loop* can also be used to automatically acquire the FID repeatedly.

Click the *Wheel Zoom* tool  at the top right of a plot to select it and scroll while hovering over the bottom axis to zoom only that axis. Zooming in on the spectrum peak will make it easier to see its shape. Alternatively the *Box Zoom* tool  can be used to select an area to zoom into. The plot view can be reset with the *Reset* button  on the toolbar.

#### **Step One**
Run the FID acquisition with the loaded shims from Part 1 (use the *Reset* button if necessary). Zoom into peak in the spectrum plot.

#### **Step Two**
Adjust the $x$ shim value to try and create the FID signal and spectrum shown in the lecture. Enter 'bad_x_shim' into the filename field and click *Save*. This will create a file called `bad_x_shim.yaml` in the User data directory shown at the top of the notebook. Include this file and the plots in your lab report. Images of the plots can be saved with the save tool  or screenshotted.

#### **Step Three**
Repeat **Step Four** for the other "bad" shims covered in the lecture.

In [7]:
# load user shims file
import yaml
SHIM_ORDER = ['shim_x', 'shim_y', 'shim_z', 'shim_z2', 'shim_zx', 'shim_zy', 'shim_xy', 'shim_x2y2']
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
if not os.path.isfile(SHIM_FILE):
    raise Exception(f'Shim file not found at {SHIM_FILE}! Ensure LAB_USER_NAME is set correctly and Part 1 has been completed.')

with open(SHIM_FILE, 'r') as f:
    SHIM_INIT = yaml.safe_load(f)

# create shim inputs, using saved user shims as initial values
shim_inputs = {}
for shim_key in SHIM_ORDER:
    shim_name = shim_key.split('_')[-1].upper()
    shim_inputs[shim_key] = pn.widgets.FloatInput(name=shim_name, start=-1, end=1, step=0.01, value=round(SHIM_INIT[shim_key], 2), width=80)

# button to reset shim inputs to the saved values
reset_btn = pn.widgets.Button(name='Reset', button_type='primary', align='end', width=100)

def reset_shim_inputs(e):
    for shim_key in SHIM_ORDER:
        shim_inputs[shim_key].value = round(SHIM_INIT[shim_key], 2)

reset_btn.on_click(reset_shim_inputs)


# input and buttons to save and load shim inputs
shim_save_name = pn.widgets.TextInput(name='', placeholder='Enter a filename, e.g. bad_x_shim')
shim_save_btn = pn.widgets.Button(name='Save', button_type='primary', width=100)
shim_load_btn = pn.widgets.Button(name='Load', button_type='primary', width=100)

def save_shims(e):
    if shim_save_name.value == '':
        return
    with open(os.path.join(LAB_DIR, f'{shim_save_name.value}.yaml'), 'w') as f:
        yaml.dump({k: v.value for k, v in shim_inputs.items()}, f) # extract values of inputs and save

def load_shims(e):
    if shim_save_name.value == '':
        return
    with open(os.path.join(LAB_DIR, f'{shim_save_name.value}.yaml'), 'r') as f:
        loaded_shims = yaml.safe_load(f)
        for shim_key in SHIM_ORDER:
            shim_inputs[shim_key].value = loaded_shims[shim_key]

shim_save_btn.on_click(save_shims)
shim_load_btn.on_click(load_shims)


from FID import FIDApp # from dashboards-inline directory that was added to sys.path
# create FID experiment using shim inputs
# set some parameters directly
override_pars = dict(
    n_scans=1,
    n_samples=4000,
    t_dw=25e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_acqdelay=50e-6,
    t_end=0.2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
fid = FIDApp(
    override_pars=override_pars,
    enable_run_loop=True
)


# display layout
pn.Column(
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    pn.Row(shim_save_name, shim_save_btn, shim_load_btn),
    fid.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] FloatInput(end=1, name='X', start=-1, step=0.01, value=-0.08, width=80)
        [1] FloatInput(end=1, name='Y', start=-1, step=0.01, value=0.14, width=80)
        [2] FloatInput(end=1, name='Z', start=-1, step=0.01, value=0.54, width=80)
        [3] FloatInput(end=1, name='Z2', start=-1, step=0.01, value=-0.16, width=80)
        [4] FloatInput(end=1, name='ZX', start=-1, step=0.01, value=0.04, width=80)
        [5] FloatInput(end=1, name='ZY', start=-1, step=0.01, value=0.13, width=80)
        [6] FloatInput(end=1, name='XY', start=-1, step=0.01, value=0.02, width=80)
        [7] FloatInput(end=1, name='X2Y2', start=-1, step=0.01, value=-0.04, width=80)
        [8] Button(align='end', button_type='primary', name='Reset', width=100)
    [1] Row
        [0] TextInput(placeholder='Enter a filename, ...)
        [1] Button(button_type='primary', name='Save', width=100)
        [2] Button(button_type='primary', name='Load', width=100)
    [2] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')